In [1]:
from sklearn import datasets
digits = datasets.load_digits()

In [30]:
import numpy as np
n_samples = len(digits.images)
X = digits.images.reshape((n_samples, -1))
y = digits.target

In [5]:
from scipy.special import softmax

W = np.random.uniform(-20, 20, size=(10, 64))
b = np. random.uniform(-20,20, size=(10))

probs = softmax(np.maximum(0, np.matmul(W,X[0])+b))
print(probs)

[1.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 1.07276881e-278 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000]


In [142]:
class Node:
    def __init__(self, name, calculate, predecessors, derives, final=False):
        self.name = name
        self.calc = calculate
        self.predecessors = list(zip(predecessors, derives))
        self.value = None
        self.gradient = None
        self.final = final

    def initialize(self):
        self.value = 0
        self.gradient = 0
        
def forward(nodes):
    order = topoSort(nodes)
    for node in order:
        # print("calculating ", node.name)
        # print([p[0].value.shape for p in node.predecessors])
        node.value = node.calc(*[p[0].value for p in node.predecessors])
    # print ("final forward ",order[-1].value.shape, order[-1].value)

def topoSortRec(node, result):
    if node in result:
        return
    for parent in node.predecessors:
        topoSortRec(parent[0], result)
    result.append(node)

def topoSort(nodes):
    result = []
    for node in nodes:
        topoSortRec(node, result)
    return result

def backpropagation(nodes):
    order = topoSort(nodes)
    for v in order:
        v.gradient = np.zeros_like(v.value)
        if v.final:
            v.gradient = 1
    for v in order[::-1]:
        values = [node[0].value for node in v.predecessors]
        for node, derive in v.predecessors:
            node.gradient += v.gradient * derive(*values)
        

In [145]:
# Define the input node
input_node = Node("X", lambda: X, [], [])

# Define the weights and biases for the hidden layer
W_hidden = Node("W_hidden", lambda: np.random.uniform(-1, 1, size=(64, 32)), [], [])
b_hidden = Node("bias_hidden", lambda: np.random.uniform(-1, 1, size=(32,)), [], [])

# Define the hidden layer node
hidden_layer = Node("hidden_layer", lambda x, W, b: np.maximum(0, np.matmul(x, W) + b), [input_node, W_hidden, b_hidden], [lambda x, W, b: None, lambda W, x, b: np.heaviside(np.matmul(x, W) + b, 0), lambda W, x, b: np.heaviside(np.matmul(x, W) + b, 0)])

# Define the weights and biases for the output layer
W_output = Node("W_output", lambda: np.random.uniform(-1, 1, size=(32, 10)), [], [])
b_output = Node("bias_output", lambda: np.random.uniform(-1, 1, size=(10,)), [], [])

# Define the output layer node
output_layer = Node("output_layer", lambda h, W, b: softmax(np.matmul(h, W) + b, axis=1), [hidden_layer, W_output, b_output], [lambda h, W, b: softmax(np.matmul(h, W) + b, axis=1), lambda W, h, b: softmax(np.matmul(h, W) + b, axis=1), lambda W, h, b: softmax(np.matmul(h, W) + b, axis=1)])

y_node = Node("y_labels", lambda: y, [], [])

# Define the cross-entropy loss node
cross_entropy_loss = Node(
    "cross_entropy_loss",
    lambda output, y: -np.sum(np.log(output[np.arange(len(y)), y])) / len(y),
    [output_layer, y_node],
    [lambda output, y: -1 / len(y) * (1 / output[np.arange(len(y)), y]), lambda output, y: None],
    True
)

# Initialize and run the forward pass
nodes = [input_node, W_hidden, b_hidden, hidden_layer, W_output, b_output, output_layer, cross_entropy_loss, y_node]
for node in nodes:
    node.initialize()

# forward(nodes)

In [146]:
# forward(nodes)
# best_loss = float('inf')
# for i in range(200000):
#     forward(nodes)
#     loss = cross_entropy_loss.value
#     if loss < 45:
#         print(i, "-> ", loss)
#     if loss < best_loss:
#         best_loss = loss
#         best_W_hidden = W_hidden.value.copy()
#         best_b_hidden = b_hidden.value.copy()
#         best_W_output = W_output.value.copy()
#         best_b_output = b_output.value.copy()

# print("Best loss:", best_loss)

# # Define a new neural network with the best parameters
# new_input_node = Node("X", lambda: X, [], [])

# new_W_hidden = Node("W_hidden", lambda: best_W_hidden, [], [])
# new_b_hidden = Node("bias_hidden", lambda: best_b_hidden, [], [])

# new_hidden_layer = Node("hidden_layer", lambda x, W, b: np.maximum(0, np.matmul(x, W) + b), [new_input_node, new_W_hidden, new_b_hidden], [lambda W, x, b: None, lambda W, x, b: np.heaviside(np.matmul(x, W) + b, 0), lambda W, x, b: np.heaviside(np.matmul(x, W) + b, 0)])

# new_W_output = Node("W_output", lambda: best_W_output, [], [])
# new_b_output = Node("bias_output", lambda: best_b_output, [], [])

# new_output_layer = Node("output_layer", lambda h, W, b: softmax(np.matmul(h, W) + b, axis=1), [new_hidden_layer, new_W_output, new_b_output], [lambda W, h, b: softmax(np.matmul(h, W) + b, axis=1), lambda W, h, b: softmax(np.matmul(h, W) + b, axis=1), lambda W, h, b: softmax(np.matmul(h, W) + b, axis=1)])

# new_y_node = Node("y_labels", lambda: y, [], [])

# new_cross_entropy_loss = Node(
#     "cross_entropy_loss",
#     lambda output, y: -np.sum(np.log(output[np.arange(len(y)), y])) / len(y),
#     [new_output_layer, new_y_node],
#     [lambda output, y: -1 / len(y) * (1 / output[np.arange(len(y)), y]), lambda output, y: None],
#     True
# )

# # Initialize and run the forward pass for the new network
# new_nodes = [new_input_node, new_W_hidden, new_b_hidden, new_hidden_layer, new_W_output, new_b_output, new_output_layer, new_cross_entropy_loss, new_y_node]
# for node in new_nodes:
#     node.initialize()

# forward(new_nodes)
# new_final_loss = new_cross_entropy_loss.value
# print("New final loss with best parameters:", new_final_loss)

# new_correct_predictions = np.sum(np.argmax(new_output_layer.value, axis=1) == y)
# new_final_accuracy = new_correct_predictions / len(y) * 100
# print("New final Accuracy: {:.2f}%".format(new_final_accuracy))

learning_rate = 0.01

for i in range(200000):
    forward(nodes)
    loss = cross_entropy_loss.value
    if loss < 45:
        print(i, "-> ", loss)
    if loss < best_loss:
        best_loss = loss
        best_W_hidden = W_hidden.value.copy()
        best_b_hidden = b_hidden.value.copy()
        best_W_output = W_output.value.copy()
        best_b_output = b_output.value.copy()

    # Backpropagation
    backpropagation(nodes)

    # Gradient descent update
    for node in nodes:
        if node.name in ["W_hidden", "bias_hidden", "W_output", "bias_output"]:
            node.value -= learning_rate * node.gradient

print("Best loss:", best_loss)

ValueError: operands could not be broadcast together with shapes (1797,10) (1797,) (1797,10) 